# Практическое занятие 8
## Вариант 11

## Создадим класс с методами

In [34]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd


class Function:
    """
    Класс Function реализует различные методы многокритериальной оптимизации.
    Включает методы для решения задачи с разными критериями и ограничениями.
    """
    def __init__(self):
        """
        Инициализация класса с определением основных ограничений задачи.
        Ограничения заданы в виде неравенств.
        """
        self.constraints = (
            {'type': 'ineq', 'fun': lambda x: 4 * x[0] - x[1]},      # 4x - y >= 0
            {'type': 'ineq', 'fun': lambda x: - x[0] + x[1] + 3},   # -x + y <= 3
            {'type': 'ineq', 'fun': lambda x: 2 * x[0] - 3 * x[1] + 6} # 2x - 3y <= 6
        )
        self.bounds = [(0, None), (0, None)] # x >= 0, y >= 0

    @staticmethod
    def Z1(x):
        """
        Целевая функция Z1.
        :param x: Вектор переменных [x, y].
        :return: Значение функции Z1 = 5x - 3y.
        """
        return 5 * x[0] - 3 * x[1]

    @staticmethod
    def Z2(x):
        """
        Целевая функция Z2.
        :param x: Вектор переменных [x, y].
        :return: Значение функции Z2 = 6x + 2y.
        """
        return 6 * x[0] + 2 * x[1]

    @staticmethod
    def Z3(x):
        """
        Целевая функция Z3.
        :param x: Вектор переменных [x, y].
        :return: Значение функции Z3 = x - 3y.
        """
        return -(x[0] - 3 * x[1])  # Минимизация вместо максимизации

    def independent_criteria_optimization(self, objective_function, initial_point=None):
        """
        Метод независимой оптимизации целевых функций.
        :param objective_function: Целевая функция для оптимизации.
        :param initial_point: Начальная точка для оптимизации (по умолчанию [0, 0]).
        :return: Результат оптимизации.
        """
        if initial_point is None:
            initial_point = [0, 0]
        result = minimize(objective_function, initial_point, method='SLSQP', bounds=self.bounds, constraints=self.constraints)
        return result

    def generalized_criterion_method(self, w1, w2, initial_point=None):
        """
        Метод обобщенного критерия. Использует веса для комбинирования критериев.
        :param w1: Вес для Z1.
        :param w2: Вес для Z2.
        :param initial_point: Начальная точка для оптимизации (по умолчанию [0, 0]).
        :return: Результат оптимизации.
        """
        w3 = 1 - w1 - w2
        if initial_point is None:
            initial_point = [0, 0]
        def objective_function(x):
            """
            Обобщенная целевая функция для метода обобщенного критерия.
            Комбинирует функции Z1, Z2 и Z3 с весовыми коэффициентами w1, w2 и w3.

            :param x: Вектор переменных [x, y], используемых в функциях Z1, Z2, Z3.
            :return: Взвешенное значение комбинации функций Z1, Z2 и Z3.
            """
            return w1 * self.Z1(x) + w2 * self.Z2(x) + w3 * self.Z3(x)

        result = minimize(objective_function, initial_point, method='SLSQP', bounds=self.bounds, constraints=self.constraints)
        return result

    def minimize_total_relative_concession(self, k1, k2, k3):
        """
        Метод минимизации общей относительной уступки.
        :param k1: Коэффициент уступки для Z1.
        :param k2: Коэффициент уступки для Z2.
        :param k3: Коэффициент уступки для Z3.
        :return: Результат оптимизации.
        """
        # Оптимизация Z1 и Z2 для нахождения m1 и m2
        m1 = self.independent_criteria_optimization(self.Z1).fun
        m2 = self.independent_criteria_optimization(self.Z2).fun
        # Оптимизация Z3 для нахождения m3 (максимум Z3)
        m3 = self.independent_criteria_optimization(self.Z3).fun

        # Новые ограничения, учитывающие m1, m2, m3, k1, k2, k3
        constraints_with_concession = (
            {'type': 'ineq', 'fun': lambda v: 4 * v[0] - v[1]},      # 4x - y >= 0
            {'type': 'ineq', 'fun': lambda v: - v[0] + v[1] + 3},   # -x + y <= 3
            {'type': 'ineq', 'fun': lambda v: 2 * v[0] - 3 * v[1] + 6}, # 2x - 3y <= 6
            {'type': 'ineq', 'fun': lambda v: self.Z1([v[0], v[1]]) - m1 + v[2] * k1},
            {'type': 'ineq', 'fun': lambda v: self.Z2([v[0], v[1]]) - m2 + v[2] * k2},
            {'type': 'ineq', 'fun': lambda v: -self.Z3([v[0], v[1]]) - m3 + v[2] * k3},
        )

        # Целевая функция для минимизации l
        def objective_function_l(v):
            """
            Целевая функция для оптимизации в методе минимизации общей относительной уступки.
            Минимизирует переменную l, которая представляет собой общую относительную уступку.

            :param v: Вектор переменных, включающий x, y и l (v[0] = x, v[1] = y, v[2] = l).
            :return: Значение переменной l, которую необходимо минимизировать.
            """
            return v[2]

            # Оптимизация l с учетом новых ограничений
        result = minimize(objective_function_l, [0, 0, 0], method='SLSQP', bounds=[(0, None), (0, None), (0, None)], constraints=constraints_with_concession)
        return result


    def sequential_concessions_method(self, delta1, delta2):
        """
        Метод последовательных уступок. Ищет оптимальное решение, делая уступки по каждому критерию.
        :param delta1: Уступка для первого критерия.
        :param delta2: Уступка для второго критерия.
        :return: Результат оптимизации.
        """
        # Этап 1: Находим максимум Z1
        m1 = self.independent_criteria_optimization(self.Z1).fun

        # Этап 2: Находим максимум Z2 с дополнительным условием на Z1
        constraints_stage_2 = self.constraints + (
            {'type': 'ineq', 'fun': lambda x: self.Z1(x) - (m1 - delta1)},
        )
        m2 = minimize(self.Z2, [0, 0], method='SLSQP', bounds=self.bounds, constraints=constraints_stage_2).fun

        # Этап 3: Находим максимум Z3 с дополнительными условиями на Z1 и Z2
        constraints_stage_3 = self.constraints + (
            {'type': 'ineq', 'fun': lambda x: self.Z1(x) - (m1 - delta1)},
            {'type': 'ineq', 'fun': lambda x: self.Z2(x) - (m2 - delta2)},
        )
        result = minimize(lambda x: -self.Z3(x), [0, 0], method='SLSQP', bounds=self.bounds, constraints=constraints_stage_3)
        return result



# Создаем экземпляр класса
function = Function()


### Метод независимой оптимизации целевых функций

In [61]:

# Оптимизация Z1
# Вызываем метод independent_criteria_optimization(метод независимой оптимизации целевых функций), который определен ранее в классе
result_Z1 = function.independent_criteria_optimization(function.Z1)

# Использование результатов Z1 для Z2
initial_point_Z2 = result_Z1.x
result_Z2 = function.independent_criteria_optimization(function.Z2, initial_point_Z2)

# Использование результатов Z1 для Z3
initial_point_Z3 = result_Z1.x
result_Z3 = function.independent_criteria_optimization(function.Z3, initial_point_Z3)


# Создание данных для DataFrame
data = {
    "Z": ["Z1", "Z2", "Z3"],
    "x": [result_Z1.x[0], result_Z2.x[0], result_Z3.x[0]],
    "y": [result_Z1.x[1], result_Z2.x[1], result_Z3.x[1]],
    "Прибыль": [result_Z1.fun, result_Z2.fun, result_Z3.fun],
    "Количество изделий": [6 * result_Z1.x[0] + 2 * result_Z1.x[1],
                           6 * result_Z2.x[0] + 2 * result_Z2.x[1],
                           6 * result_Z3.x[0] + 2 * result_Z3.x[1]],
    "Себестоимость": [result_Z1.x[0] - 3 * result_Z1.x[1],
                      result_Z2.x[0] - 3 * result_Z2.x[1],
                      result_Z3.x[0] - 3 * result_Z3.x[1]]
}

# Создание DataFrame
df1 = pd.DataFrame(data)


method_name = "Независимая оптимизация целевых функций"
df1.insert(0, 'Метод', [""] * len(df1))
df1.at[0, 'Метод'] = method_name

df1

,Метод,Z,x,y,Прибыль,Количество изделий,Себестоимость
0,Независимая оптимизация целевых функций,Z1,6.000000e-01,2.4,-4.200000e+00,8.400000e+00,-6.600000e+00
1,,Z2,1.432188e-14,0.0,8.593126e-14,8.593126e-14,1.432188e-14
2,,Z3,3.000000e+00,0.0,-3.000000e+00,1.800000e+01,3.000000e+00


### Метод обобщенного критерия

In [58]:
# Значение весов
W1 = 0.3
W2 = 0.3

result_generalized = function.generalized_criterion_method(W1, W2, initial_point_Z2)
#print("Результат оптимизации обобщенного критерия:", result_generalized)

data = {
    "Z": [" - "],
    "x": [result_generalized.x[0]],
    "y": [result_generalized.x[1]],
    "Прибыль": [result_generalized.fun],
    "Количество изделий": [6 * result_generalized.x[0] + 2 * result_generalized.x[1]],
    "Себестоимость": [result_generalized.x[0] - 3 * result_generalized.x[1]]
}

df2 = pd.DataFrame(data)

method_name = "Обобщенный критерий"
df2.insert(0, 'Метод', [""] * len(df2))
df2.at[0, 'Метод'] = method_name

df2


,Метод,Z,x,y,Прибыль,Количество изделий,Себестоимость
0,Обобщенный критерий,-,0.0,0.0,0.0,0.0,0.0


### Метод минимизации общей относительной уступки.

In [57]:
# Примерные значения коэффициентов
K1 = 5
K2 = 2
K3 = 2
result_concession = function.minimize_total_relative_concession(K1, K2, K3)
#print("Результат минимизации общей относительной уступки:", result_concession)

data = {
    "Z": [" - "],
    "x": [result_concession.x[0]],
    "y": [result_concession.x[1]],
    "Прибыль": [result_concession.fun],
    "Количество изделий": [6 * result_concession.x[0] + 2 * result_concession.x[1]],
    "Себестоимость": [result_concession.x[0] - 3 * result_concession.x[1]]
}

df3 = pd.DataFrame(data)
method_name = "Минимизация общей относительной уступки"
df3.insert(0, 'Метод', [""] * len(df3))
df3.at[0, 'Метод'] = method_name

df3


,Метод,Z,x,y,Прибыль,Количество изделий,Себестоимость
0,Минимизация общей относительной уступки,-,0.0,0.0,0.0,0.0,0.0


### Метод последовательных уступок

In [56]:
# Дельты
DELTA1 = 5
DELTA2 = 2

result_sequential = function.sequential_concessions_method(DELTA1, DELTA2)
#print("Результат метода последовательных уступок:", result_sequential)


data = {
    "Z": [" - "],
    "x": [result_sequential.x[0]],
    "y": [result_sequential.x[1]],
    "Прибыль": [result_sequential.fun],
    "Количество изделий": [6 * result_sequential.x[0] + 2 * result_sequential.x[1]],
    "Себестоимость": [result_sequential.x[0] - 3 * result_sequential.x[1]]
}

df4 = pd.DataFrame(data)

method_name = "Последовательные уступки"
df4.insert(0, 'Метод', [""] * len(df4))
df4.at[0, 'Метод'] = method_name


df4


,Метод,Z,x,y,Прибыль,Количество изделий,Себестоимость
0,Последовательные уступки,-,15.0,12.0,-21.0,114.0,-21.0


In [73]:
df = pd.concat([df1, df2, df3, df4])


pd.set_option('display.float_format', '{:.2f}'.format)
df = df.round(2)

df

,Метод,Z,x,y,Прибыль,Количество изделий,Себестоимость
0,Независимая оптимизация целевых функций,Z1,0.60,2.40,-4.20,8.40,-6.60
1,,Z2,0.00,0.00,0.00,0.00,0.00
2,,Z3,3.00,0.00,-3.00,18.00,3.00
0,Обобщенный критерий,-,0.00,0.00,0.00,0.00,0.00
0,Минимизация общей относительной уступки,-,0.00,0.00,0.00,0.00,0.00
0,Последовательные уступки,-,15.00,12.00,-21.00,114.00,-21.00


### Общий вывод



1. **Независимая оптимизация целевых функций**:
   - Для минимизации прибыли (`Z1`), наилучшие параметры: `x = 0.60`, `y = 2.40`.
   - Оптимизация количества изделий (`Z2`) приводит к нулевым значениям.
   - Для максимизации себестоимости (`Z3`), оптимальными оказались параметры: `x = 3.00`, `y = 0.00`.

2. **Обобщенный критерий** и **Минимизация общей относительной уступки**:
   - Оба метода приводят к нулевым значениям по всем параметрам.

3. **Последовательные уступки**:
   - Метод дает значительные параметры `x = 15.00` и `y = 12.00`, и ведет к значительным убыткам по прибыли и себестоимости.